In [15]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
Dependências instaladas.
API do Google Earth Engine já inicializada.
Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado em /content/drive


In [16]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
import geemap

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [17]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2023-12-31'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2023-12-31
Área de Interesse: Bioma Cerrado


In [52]:
# ==============================================================================
# PASSO 2: CARREGAR ASSETS DE FOCOS DE CALOR
# ==============================================================================
print("Iniciando o carregamento dos assets de focos de calor...")

# 1. Definição caminho-base (pasta) onde os assets estão.
BASE_PATH = 'projects/the-byway-476116-n7/assets/'

# 2. Crie uma lista com os nomes de todos os seus assets anuais.
asset_names = [
    'bdqueimadas_2014-01-01_2014-12-31',
    'bdqueimadas_2015-01-01_2015-12-31',
    'bdqueimadas_2016-01-01_2016-12-31',
    'bdqueimadas_2017-01-01_2017-12-31',
    'bdqueimadas_2018-01-01_2018-12-31',
    'bdqueimadas_2019-01-01_2019-12-31',
    'bdqueimadas_2020-01-01_2020-12-31',
    'bdqueimadas_2021-01-01_2021-12-31',
    'bdqueimadas_2022-01-01_2022-12-31',
    'bdqueimadas_2023-01-01_2023-12-31'
]

# 3. Crie uma lista VAZIA para armazenar os objetos GEE
list_of_collections = []

# 4. Iteração pela lista de nomes para carregar cada asset
print(f"Encontrados {len(asset_names)} nomes de assets. Carregando do GEE...")

for name in asset_names:
    # Constrói o caminho completo do asset
    full_asset_path = BASE_PATH + name

    # Carrega o asset do GEE como uma FeatureCollection
    fc = ee.FeatureCollection(full_asset_path)

    # Adiciona a coleção carregada à nossa lista
    list_of_collections.append(fc)

print(f"Carregou {len(list_of_collections)} coleções de features com sucesso.")

# 5. Junção de todas as coleções em uma única e grande FeatureCollection
active_fires_collection = ee.FeatureCollection(list_of_collections).flatten()
print(active_fires_collection.first().getInfo())

print("\nAssets anuais de focos de calor mesclados com sucesso no servidor!")
print("O objeto 'active_fires_collection' está pronto.")

Iniciando o carregamento dos assets de focos de calor...
Encontrados 10 nomes de assets. Carregando do GEE...
Carregou 10 coleções de features com sucesso.
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-44.80399949627662, -11.890001217668136]}, 'id': '0_000000000000000001b7', 'properties': {'Bioma': 'Cerrado', 'DataHora': '2014/01/18 16:39:00', 'DiaSemChuv': 3, 'Estado': 'BAHIA', 'FRP': 8.8, 'Latitude': -11.89, 'Longitude': -44.804, 'Municipio': 'ANGICAL', 'Pais': 'Brasil', 'Precipitac': 1.6, 'RiscoFogo': 0.6, 'Satelite': 'AQUA_M-T'}}

Assets anuais de focos de calor mesclados com sucesso no servidor!
O objeto 'active_fires_collection' está pronto.


In [82]:
# ==============================================================================
# PASSO 3: CARREGAMENTO DAS COLEÇÕES DE FEATURES
# ==============================================================================

# --- 3.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 3.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- .4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2')
print(land_cover.bandNames().getInfo())

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87624 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3630 imagens de LST (diárias).
Dados de elevação e declive carregados.
['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_

In [79]:
# ==============================================================================
# PASSO 4: ALINHAMENTO ESPAÇO-TEMPORAL (ENRIQUECIMENTO DE DADOS)
# ==============================================================================

# --- 4.1 Amostragem de Preditores Estáticos (Topografia) ---
# (Alinhamento puramente espacial, pois os dados não mudam com o tempo)

print("Iniciando Passo 4.1: Alinhamento Estático (Topografia)...")

# 1. Renomear as bandas (do Passo 3.3) para nomes de colunas claros
#    (A 'elevation' já está selecionada, 'slope' é calculado)
elevation_renamed = elevation.rename('elevation_static')
slope_renamed = slope.rename('slope_static')

# 2. Juntar as duas imagens em uma única imagem "estática" com 2 bandas
static_predictors = ee.Image.cat([
    elevation_renamed,
    slope_renamed
])

# 3. Amostrar os valores dessas bandas para CADA PONTO do 'active_fires_collection'
#    Esta função vai ADICIONAR as novas colunas ('elevation_static' e 'slope_static')
#    a cada ponto de fogo.
dataset_with_static = static_predictors.sampleRegions(
    collection=active_fires_collection,  # Nossa FeatureCollection do Passo 2
    properties=active_fires_collection.first().propertyNames(), # Manter TODAS as colunas do INPE
    scale=TARGET_RESOLUTION,             # Resolução de 30m (do Passo 1)
    geometries=True                      # Manter a geometria (o ponto)
)

print("Alinhamento estático concluído.")
print("A variável 'dataset_with_static' agora contém:")
print("Pontos de Fogo (INPE) + Colunas de 'elevation_static' e 'slope_static'")

Iniciando Passo 4.1: Alinhamento Estático (Topografia)...
Alinhamento estático concluído.
A variável 'dataset_with_static' agora contém:
Pontos de Fogo (INPE) + Colunas de 'elevation_static' e 'slope_static'


In [80]:
# ==============================================================================
# TESTE DA LÓGICA 4.1 (O "TESTE DE VERDADE" - RÁPIDO)
# ==============================================================================
import pprint
print("Iniciando Teste da Lógica 4.1 (com 10 pontos de 2014)...")

try:
    # 1. Crie uma amostra de teste REALMENTE LEVE
    #    Carrega SÓ o asset de 2014 e pega 10 pontos.
    #    Isso é instantâneo.
    asset_2014_path = BASE_PATH + asset_names[0] # Pega o primeiro nome da lista ('...2014...')
    test_collection = ee.FeatureCollection(asset_2014_path).limit(10)

    print("  Amostra de 10 pontos de 2014 carregada.")

    # 2. Recrie as imagens estáticas (globais, sem clip - RÁPIDO)
    elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
    slope = ee.Terrain.slope(elevation)

    elevation_renamed = elevation.rename('elevation_static')
    slope_renamed = slope.rename('slope_static')
    static_predictors = ee.Image.cat([elevation_renamed, slope_renamed])

    print("  Imagens estáticas (globais) prontas.")

    # 3. Aplique a MESMA LÓGICA do Passo 4.1, mas na coleção de teste LEVE
    test_result = static_predictors.sampleRegions(
        collection=test_collection,  # <-- Usando a coleção LEVE (10 pontos de 2014)
        properties=test_collection.first().propertyNames(), # Pegando as propriedades da coleção LEVE
        scale=TARGET_RESOLUTION,
        geometries=True
    )

    # 4. Chame o .getInfo() no RESULTADO do teste
    print("\nTeste Concluído. Inspecionando o primeiro ponto da amostra:")
    pprint.pprint(test_result.first().getInfo())

    print("\n>>> SUCESSO! As novas colunas 'elevation_static' e 'slope_static' devem estar visíveis.")

except Exception as e:
    print(f"\n>>> FALHA NO TESTE: {e}")

Iniciando Teste da Lógica 4.1 (com 10 pontos de 2014)...
  Amostra de 10 pontos de 2014 carregada.
  Imagens estáticas (globais) prontas.

Teste Concluído. Inspecionando o primeiro ponto da amostra:
{'geometry': {'coordinates': [-44.804003039250375, -11.89010192133555],
              'geodesic': False,
              'type': 'Point'},
 'id': '000000000000000001b7_0',
 'properties': {'Bioma': 'Cerrado',
                'DataHora': '2014/01/18 16:39:00',
                'DiaSemChuv': 3,
                'Estado': 'BAHIA',
                'FRP': 8.8,
                'Latitude': -11.89,
                'Longitude': -44.804,
                'Municipio': 'ANGICAL',
                'Pais': 'Brasil',
                'Precipitac': 1.6,
                'RiscoFogo': 0.6,
                'Satelite': 'AQUA_M-T',
                'elevation_static': 436,
                'slope_static': 1.3258910179138184},
 'type': 'Feature'}

>>> SUCESSO! As novas colunas 'elevation_static' e 'slope_static' devem esta

In [ ]:
# ==============================================================================
# PASSO 4.2: ALINHAMENTO ESPAÇO-TEMPORAL (Anual - LULC)
# ==============================================================================
print("Iniciando Passo 4.2: Alinhamento Anual (Cobertura do Solo)...")

# A 'dataset_with_static' é a nossa "receita" do Passo 4.1
# A 'land_cover' é a imagem com bandas anuais (do Passo 3.4)

# 1. Definimos uma função que será aplicada a CADA ponto de fogo
def sample_annual_lulc(feature):
    # 'feature' é o nosso ponto de fogo (que já tem elev/slope)

    # 2. Pega a data do fogo (ex: '2014/01/18 16:39:00')
    date_string = ee.String(feature.get('DataHora'))

    # 3. Extrai o ano (os primeiros 4 caracteres)
    year_string = date_string.slice(0, 4) # Resultado: '2014'

    # 4. Cria o nome da banda do LULC correspondente
    #    ex: 'classification_' + '2014' = 'classification_2014'
    lulc_band_name = ee.String('classification_').cat(year_string)

    # 5. Seleciona APENAS essa banda da imagem de LULC
    lulc_image_this_year = land_cover.select(lulc_band_name)

    # 6. Amostra o valor dessa banda NO PONTO do 'feature'
    #    O .first() pega o resultado da amostragem (que é uma FeatureCollection)
    lulc_value = lulc_image_this_year.sample(
        region=feature.geometry(), # O Ponto
        scale=TARGET_RESOLUTION,   # 30m
        numPixels=1                # Pega só 1 pixel
    ).first() # Pega o primeiro (e único) resultado

    # 7. Adiciona o valor como uma nova propriedade
    #    Usamos 'ee.Algorithms.If' para evitar erros se o pixel for nulo
    return feature.set('lulc_class', ee.Algorithms.If(
        lulc_value,                                # Se o valor existir...
        lulc_value.get(lulc_band_name),            # ...pegue o valor
        -9999                                      # ...senão, use -9999 (NoData)
    ))

# 8. Aplica a função a TODOS os pontos da nossa coleção
#    Esta é a nova "receita" principal, agora com dados estáticos + LULC
dataset_with_lulc = dataset_with_static.map(sample_annual_lulc)

print("Alinhamento anual (LULC) definido.")
print("A variável 'dataset_with_lulc' agora contém:")
print("Pontos + Topografia + lulc_class")

# (Não vamos testar aqui, pois a 'dataset_with_static' é muito pesada)